# Detetion Rates of sGRBs 

This notebook explore the detection rates of sGRBs by the Swift satellite.
To do so, we utilise a form of Monte Carlo simulation to simulate a set of sGRBs, we then compare these to the known detected sGRBs.

## Importing and cleaning the Swfit data   

In [ ]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import Planck18 as cosmo
from astropy import units as u
from scipy.integrate import quad


We will start by importing and sorting through GRB data collected by the Swift Observatory (https://swift.gsfc.nasa.gov/archive/grb_table).
We are going to be importing two data sets, one of all GRB under 2 seconds and the other of all GRB containing Redshift information. Its going to clean the data and filter what we need. Currently, it takes out all the sGRBs without redshift information and exports them to a separate file, this can be used for manually adding redshift data after the fact. It also exports a combined table containing all the sGRBs with redshift information based on the intrinsic time of the sGRBs. 

Conditions:
Filters out GRB that do not contain redshift information
calculates $T_{Intrinsic}$
filters GRB $\leq$ 2 seconds

In [ ]:
# Function to clean numeric columns and keep the first number NEEDS TO BE CHANGED!!!!!!!
def clean_numeric_column(df, column_name):
    # Extract the first number from each entry
    df[column_name] = df[column_name].astype(str).apply(
        lambda x: re.findall(r'[\d.]+', x)[0] if re.findall(r'[\d.]+', x) else None)
    return df

# Function to convert columns to numeric values
def convert_columns_to_numeric(df, columns_to_exclude):
    for col in df.columns:
        if col not in columns_to_exclude:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df


# Function to calculate T_intrinsic
def calculate_T_intrinsic(df, duration_col, redshift_col):
    df['T_intrinsic'] = df[duration_col] / (1 + df[redshift_col])
    return df

def split_and_update_photon_index(df):
    def split_photon_index(row):
        # Convert row to string and split by comma
        parts = str(row).split(',')

        # Extract and clean up the number and text parts
        number_part = parts[0].strip() if len(parts) > 0 else ''
        text_part = parts[1].strip() if len(parts) > 1 else ''

        return number_part, text_part

    # Apply the function to split the column and add the results as new columns
    df[['BAT Photon Index Value', 'BAT Photon Index Type']] = df['BAT Photon Index (15-150 keV) (PL = simple power-law, CPL = cutoff power-law)'].apply(split_photon_index).apply(pd.Series)

    # Drop the original column
    df.drop('BAT Photon Index (15-150 keV) (PL = simple power-law, CPL = cutoff power-law)', axis=1, inplace=True)
    return df



In [ ]:
# Load data
grb_table_rf = pd.read_csv('Data/grb_table_rf.txt', sep='\t')
grb_table = pd.read_csv('Data/grb_table.txt', sep='\t')


# Clean and convert 'Redshift' column in grb_table_rf
grb_table_rf = clean_numeric_column(grb_table_rf, 'Redshift')
grb_table = clean_numeric_column(grb_table, 'Redshift')

# Split the photon index column into two columns
grb_table_rf = split_and_update_photon_index(grb_table_rf)
grb_table = split_and_update_photon_index(grb_table)

# List of columns to keep as strings
columns_to_exclude = [
    'GRB',
    'BAT Photon Index Type',
    'XRT RA (J2000)',
    'XRT Dec (J2000)'
]

# Convert the columns to numeric values
grb_table_rf = convert_columns_to_numeric(grb_table_rf, columns_to_exclude)
grb_table = convert_columns_to_numeric(grb_table, columns_to_exclude)


In [ ]:
# Calculate T_intrinsic for grb_table_rf and grb_table
grb_table_rf = calculate_T_intrinsic(grb_table_rf, 'BAT T90 [sec]', 'Redshift')
grb_table = calculate_T_intrinsic(grb_table, 'BAT T90 [sec]', 'Redshift')

# Filter both datasets for T_intrinsic times of 2 seconds or less
filtered_grb_table_rf = grb_table_rf[grb_table_rf['T_intrinsic'] <= 2]
filtered_grb_table = grb_table[grb_table['T_intrinsic'] <= 2]

# Separate sGRBs without redshift information from grb_table and export
sgrbs_without_redshift = grb_table[grb_table['Redshift'].isna()]

# Merge the filtered datasets
combined_grb_table = pd.concat([filtered_grb_table_rf, filtered_grb_table], ignore_index=True)

# Removing potential duplicates
combined_grb_table = combined_grb_table.drop_duplicates()

# Reviewing and ensuring all columns are included after merge
print("Columns in the combined dataset:", combined_grb_table.columns)


In [ ]:
# Exporting the combined dataset
output_file_path = 'Data/combined_grb_table.txt'
sgrbs_without_redshift.to_csv(output_file_path, sep='\t', index=False)
combined_grb_table.to_csv(output_file_path, sep='\t', index=False)
print(f"Combined GRB data exported to {output_file_path}")
print(f"Total Number of sGRB: {len(combined_grb_table)}")

## Simulating sGRBs

Now we have the data we need, we can start simulating sGRBs. We will be using a Monte Carlo simulation to simulate a set of sGRBs.

For the parameters we will have teh following parameters:

1. **Redshift** 
    
        Our redshift is drawn from a probalbility distribtion. 

2. **Sky Position**

3. **Interval between sGRBs**

4. **Mass**

5. **Beaming Angle*

6. **Luminosity**

        $$\Phi(L_p) \propto \begin{cases} 
        \left(\frac{L_p}{L_*}\right)^\alpha & \text{if } L_*/|\Delta| < L_p \leq L_*, \\
        \left(\frac{L_p}{L_*}\right)^\beta & \text{if } L_* < L_p \leq \Delta L_*, 
        \end{cases}$$
    
7. **Log of the intrinsic time**

We will be using the redshift distribution of sGRBs to simulate the redshift of the sGRBs. We will be using the duration distribution of sGRBs to simulate the duration of the sGRBs. We will be using the luminosity distribution of sGRBs to simulate the luminosity of the sGRBs. We will be using the flux distribution of sGRBs to simulate the flux of the sGRBs. We will be using the sky distribution of sGRBs to simulate the sky location of the sGRBs. We will be using the photon index distribution of sGRBs to simulate the photon index of the sGRBs. We will be using the fluence distribution of sGRBs to simulate the fluence of the sGRBs. We will be using the peak energy distribution of sGRBs to simulate the peak energy of the sGRBs.

### Redshift Probability Distributions
The redshift is drawn from a probability distribution.




In [ ]:
zb = np.arange(0, 10.01, 0.01)

Luminosity_swift = calculate_luminosity(zb, Flim, H, omega, alpha_min, beta_min, Eb_min, erg_swift_min, photons_swift_min)
sensitivity_swift = compute_sensitivity(Luminosity_swift, L_star, delta1, delta2, alpha_mean, beta_mean)

# File paths for uploaded data files
data_files = [
    'Mathematica_code/Data_Prob/Pz_Rh1_100.txt',
    'Mathematica_code/Data_Prob/Pz_Rh1_1000.txt',
    'Mathematica_code/Data_Prob/Pz_Rc1_100.txt',
    'Mathematica_code/Data_Prob/Pz_Rc1_1000.txt',
    'Data/Pz_Rc2_100.txt',
    'Data/Pz_Rc2_1000.txt',
    'Data/Pz_Rome_100.txt',
    'Data/Pz_Rome_1000.txt'
]

# Labels for the data files
labels = [
    'Pz H06 100',
    'Pz H06 1000',
    'Pz Rc1 100',
    'Pz Rc1 1000',
    'Pz Rc2 100',
    'Pz Rc2 1000',
    'Pz Rome 100',
    'Pz Rome 1000'
]


# Plotting
plt.figure(figsize=(10, 8))

# Plot data from each file
for file, label in zip(data_files, labels):
    data = pd.read_csv(file, header=None, sep='\s+')
    pz = data[1].values * sensitivity_swift
    plt.plot(zb, pz, label=label, linewidth=2)

plt.grid()
plt.xlabel('Redshift', fontsize=16)
plt.ylabel(' ', fontsize=16)
plt.legend()
plt.show()
